# Kaggle Project - Breast Cancer Wisconsin (Diagnostic) Data Set
# Part 2 - Reproduce 1994 Legacy Model with Support Vector Machine

## Summary:
This section reproduces the classificaiton model described in the 1994 research paper <em>[Breast Cancer Diagnosis and Prognosis via Linear Programming](https://www.semanticscholar.org/paper/Breast-Cancer-Diagnosis-and-Prognosis-Via-Linear-Mangasarian-Street/3721bb14b16e866115c906336e9d70db096c05b9)</em> published/revised by Olvi L. Mangasarian, W. Nick Street & William H. Wolberg.

As shown in **Part 1 - EDA**, total 30 features were computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. Based on the mathematical formula and loss function described in the paper, the diagnostic model, <em>multisurface method Tree (MSM-T)</em>, is analogous to **[Support Vector Machine](https://en.wikipedia.org/wiki/Support-vector_machine)**. The best results were obtained with one plane the the following three features:

1. extreme area, 
2. extreme smoothness, and
3. mean texture.

The accuracy of the model estimated with cross-validation was 97.5%. The paper then used <em>Parzen window kernel</em> technique (the mechanism is analogus to DBSCAN) to construct the probability density function for benign and malignant along the separating plane.

This part will reproduce the model for the diagnostic portion of the paper and reconstruct the probability density.

## Step 1 - Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve, recall_score

%matplotlib inline

## Step 2 - Import Data

In [ ]:
# Import from data.csv
df = pd.read_csv('../data/data.csv')
# Data and features
print(f'df dimension: {df.shape}')
# Check first 5 rows of data
df.head()

## Step 3 - Feature Selection

### 3.1 - Categorize target variable "diagnosis" for visual exploration

In [ ]:
# Create dummy variable for diagnosis
df = pd.get_dummies(df, columns=['diagnosis'], drop_first=True)

# Check data inbalance
df['diagnosis_M'].value_counts()

### 3.2 - 3D Plot

In [ ]:
# 3D Scatter plot for three modeled features
fig = px.scatter_3d(df, 
                    x='texture_mean', 
                    y='area_worst', 
                    z='smoothness_worst', 
                    color='diagnosis_M',
                    size='area_worst'
                   )
fig.show();

### 3.3 - Heatmap

In [ ]:
# Heatmap color scheme
heatmap_scheme = sns.diverging_palette(220, 10, sep=80, n=100)

# Heatmap for selected features
plt.figure(figsize=(6, 5))
# Figure - Show correlation between same features in different groups (e.g., mean, se, and worst)
sns.heatmap(df[['texture_mean','area_worst','smoothness_worst']].corr(), 
            annot=True, annot_kws=None, fmt=".2f",                                   # Annotate format
            cmap=heatmap_scheme, vmin=-1, vmax=1,                                    # Color bar scale
            linewidths=0.1, linestyle='-.' )                                         # Gridline format
plt.title('Correlation - Selected Features', fontsize=13, pad=10)
plt.xticks(rotation=45);

### 3.4 - X and y

In [ ]:
feat_legacy = ['texture_mean', 'area_worst', 'smoothness_worst']
X = df[feat_legacy]
y = df['diagnosis_M']

## Step 4 - Model

### 4.1 - Pipeline & GridSearch

In [ ]:
# Instantiate pipeline
pipe = Pipeline([
    ('sc', StandardScaler()),
    ('svc', SVC(random_state=42, probability=True))
])

# Model parameters for GridSearch
param_grid = {
    'svc__kernel': ['rbf'],
    'svc__C': np.logspace(-3, 3, 7),
    'svc__gamma': np.logspace(-3, 3, 7)
}

# Instantiate GridSearch
search = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)

# Fit model
search.fit(X, y)
print(f'The best parameters are: {search.best_params_}.')

# Predit Train
y_pred = search.predict(X)

## 5.0 Model Evaluation

### 5.1 Metrics

In [ ]:
print(f'The prediction accuracy, estimated with cross-validation is {round(search.best_score_*100, 1)}%.')
print(f'The training recall is {round(recall_score(y, y_pred)*100, 1)}%.')
print(f'The training F-score is {round(f1_score(y, y_pred)*100, 1)}%.')

### 5.2 Visualization

In [ ]:
# Calculate the probability of 
m_prob = [i[1] for i in search.predict_proba(X)]

pred_df = pd.DataFrame({'true_y': y, 'pred_y': m_prob})

**Probability Distribution**

In [ ]:
# Create figure.
plt.figure(figsize = (10,7))

# Create two histograms of observations.
plt.hist(pred_df[pred_df['true_y'] == 0]['pred_y'],
         bins=25,
         color='b',
         alpha = 0.6,
         label='Diagnosis = Benign')
plt.hist(pred_df[pred_df['true_y'] == 1]['pred_y'],
         bins=25,
         color='red',
         alpha = 0.6,
         label='Diagnosis = Malignant')

# Label axes.
plt.title('Probability Distribution of Prediction = Maglignant', fontsize=18, pad=20)
plt.ylabel('Frequency', fontsize=14)
plt.xlabel('Predicted Probability of Malignant', fontsize=14)

# Create legend.
plt.legend(fontsize=20);

In [ ]:
# Check the model output probability the top 5 misclassfied maglignant cells
pred_df[pred_df['true_y'] == 1].sort_values(by='pred_y').head(5)

In [ ]:
# Confusion Matris
pd.DataFrame(confusion_matrix(y, y_pred), columns=['pred_N', 'pred_T'], index=['actual_N', 'actual_T'])

The prediction accuracy of the model based on cross-validation is 97.5%, which matches that in the 1994 paper. As shown in the probability distribution figure above, for certain maglignant cells, the probability being correctly classified is low (nearly 0). This implies the similarity of the chose features between certain cancerous cell and healthy cell and there is not enough info for the model to distinguish between the two. To minimize the false negative diagnosis, we have to set the probability threshold to 0, or simply assume all cells are malignant (which is not a very useful model). The proposed solution to improve the model is to add more features, which will be discussed in **Part 3**.

**ROC Score & ROC AUC Curve**

In [ ]:
# Create figure.
plt.figure(figsize = (10,7))

# Create threshold values.
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)

# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(pred_df, 'true_y', 'pred_y', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_y', 'pred_y', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title('Receiver Operating Characteristic Curve', fontsize=18, pad=20)
plt.ylabel('Sensitivity', fontsize=14)
plt.xlabel('1 - Specificity', fontsize=14)

# Create legend.
plt.legend(fontsize=12);

Though the ROC AUC curve shows the model is accurage. For medical diagnosis, the priority is to minimize the false negative. Seeking for further improvement is necessary.

**Evaluation of C & Gamma - Heatmap**

In [ ]:
# Plot the 
C = np.logspace(-3, 3, 7)
gamma = np.logspace(-3, 3, 7)
val_scores = search.cv_results_['mean_test_score'].reshape(len(C),len(gamma))

### The following code in this cell is from Sklearn documentary
# https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#rbf-svm-parameters

# Utility function to move the midpoint of a colormap to be around
# the values of interest.

class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Draw heatmap of the validation accuracy as a function of gamma and C
#
# The score are encoded as colors with the hot colormap which varies from dark
# red to bright yellow. As the most interesting scores are all located in the
# 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
# as to make it easier to visualize the small variations of score values in the
# interesting range while not brutally collapsing all the low score values to
# the same color.
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=0, right=1.0, bottom=0, top=1.0)
plt.imshow(val_scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma)), gamma, rotation=45)
plt.yticks(np.arange(len(C)), C)
plt.title('Validation accuracy')
plt.show()

As shown in the heatmap abbove, the model performs best along the diagonal when the value of `gamma` is low and that of `C` is high. 

<span style="color:blue">*Intuitively, the `gamma` parameter defines how far the influence of a single training example reaches, with low values meaning ‘far’ and high values meaning ‘close’. The `gamma` parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.
The `C` parameter trades off correct classification of training examples against maximization of the decision function’s margin. For larger values of `C`, a smaller margin will be accepted if the decision function is better at classifying all training points correctly. A lower `C` will encourage a larger margin, therefore a simpler decision function, at the cost of training accuracy. In other words`C` behaves as a regularization parameter in the SVM.*</span>

Above text directly cited from Sklearn documentary. As shown above, once `gamma` becomes too large, the radius of the influece of the support vector becomes so small that regularization can no longer help prevent overfitting. On the other hand, too much regularization (small `C`) makes the model insensitive to the variation of `gamma`.

**3D Plot**

In [ ]:
# Add prediction to the original dataframe
df['diagnosis_M_Pred'] = y_pred

# Relabel class for 3D-Plot 
# benigh = 3, false negative = 2, falst positive = 1, malignant = 0
y_compare=[]
for index, diag in enumerate(df['diagnosis_M']):
    if (diag != df['diagnosis_M_Pred'][index]) & (diag == 0):
        y_compare.append(int(2))
    elif (diag != df['diagnosis_M_Pred'][index]) & (diag == 1):
        y_compare.append(int(1))
    elif diag == 0:
        y_compare.append(int(3))
    else:
        y_compare.append(int(0))
df['diag_compare'] = y_compare

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=df['texture_mean'], 
    y=df['area_worst'], 
    z=df['smoothness_worst'] ,
    mode='markers',
    marker=dict(
        size=4,
        color=df['diag_compare'], # set color to an array/list of desired values
        colorscale= 'rainbow',   # choose a colorscale
        colorbar=None,
        opacity=0.6
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

The 3D-plot locates the misclassfied cell in purple and orange colors. As discussed above, the model failed to correctly classfy these due to the similarity of the chose features between the benigh and maglignant cells. 